In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_nhanes(balanced=True)

Dataset shape: (2000, 95)
33.65% missing data
Class distribution: (array([1., 2.]), array([1000, 1000]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 1),
                       (46, 9, 5),
                       (96, 11, 1),
                       (96, 13, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 1.84 sec - Loss 0.060620 - ACC 62.45% - ACC Mean 62.45% - AUC 68.32% - AUC Mean 68.32% - Deter 000
Ite 00050 - 1.04 sec - Loss 0.011664 - ACC 61.70% - ACC Mean 62.30% - AUC 67.32% - AUC Mean 67.39% - Deter 007
Ite 00093 - 1.01 sec - Loss 0.007196 - ACC 61.50% - ACC Mean 61.89% - AUC 66.16% - AUC Mean 67.00% - Deter 050
Early stop ite 93, rollback to correction of ite 43, whith acc of 64.3% and auc of 69.45%
OUR METHOD RUN 1/10 - acc: 64.3% +- 0.0% - auc: 69.45% +- 0.0%
Ite 00000 - 0.95 sec - Loss 0.145551 - ACC 63.60% - ACC Mean 63.60% - AUC 68.19% - AUC Mean 68.19% - Deter 000
Ite 00050 - 1.01 sec - Loss 0.054923 - ACC 61.60% - ACC Mean 62.31% - AUC 67.23% - AUC Mean 67.45% - Deter 022
Ite 00100 - 1.14 sec - Loss 0.037251 - ACC 61.60% - ACC Mean 62.43% - AUC 66.36% - AUC Mean 67.86% - Deter 027
Ite 00123 - 0.93 sec - Loss 0.032460 - ACC 61.70% - ACC Mean 62.37% - AUC 67.54% - AUC Mean 67.70% - Deter 050
Early stop ite 123, rollback to correction of ite 73, whith acc of 63.

## Run Median, Mean and KNN

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
knn_imputer = KNNImputer(n_neighbors=10, weights='uniform')

median_completed = median_imputer.fit_transform(data_missing_nans)
mean_completed = mean_imputer.fit_transform(data_missing_nans)
knn_completed = knn_imputer.fit_transform(data_missing_nans)

## All runs would be the same since deterministic methods
scores = utils.get_scores(median_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEDIAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
median_accs = acc
median_aucs = auc

scores = utils.get_scores(mean_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mean_accs = acc
mean_aucs = auc

scores = utils.get_scores(knn_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'KNN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
knn_accs = acc
knn_aucs = auc

MEDIAN - acc: 60.3% +- 0.0% - auc: 65.11% +- 0.0%
MEAN - acc: 60.35% +- 0.0% - auc: 66.1% +- 0.0%
KNN - acc: 63.0% +- 0.0% - auc: 67.78% +- 0.0%


## Run GAIN

In [6]:
gain_parameters = {'batch_size': 128,
                   'hint_rate': .9,
                   'alpha': 100,
                   'iterations': 5000}

gain_accs, gain_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = gain(data_missing_nans, gain_parameters)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nGAIN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    gain_accs.append(acc.mean())
    gain_aucs.append(auc.mean())
print(f'GAIN GLOBAL - acc: {round(np.array(gain_accs).mean() * 100, 2)}% +- {round(np.array(gain_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(gain_aucs).mean() * 100, 2)}% +- {round(np.array(gain_aucs).std() * 100, 2)}%')

100%|██████████| 5000/5000 [00:58<00:00, 85.36it/s]



GAIN RUN 1/10 - acc: 61.3% +- 2.52% - auc: 67.22% +- 3.64%



100%|██████████| 5000/5000 [00:58<00:00, 84.99it/s]



GAIN RUN 2/10 - acc: 61.3% +- 2.5% - auc: 66.84% +- 2.99%



100%|██████████| 5000/5000 [00:58<00:00, 85.36it/s]


...


GAIN RUN 8/10 - acc: 62.2% +- 2.47% - auc: 67.05% +- 3.47%



100%|██████████| 5000/5000 [00:57<00:00, 87.36it/s]



GAIN RUN 9/10 - acc: 60.95% +- 1.33% - auc: 66.85% +- 1.68%



100%|██████████| 5000/5000 [00:58<00:00, 85.49it/s]



GAIN RUN 10/10 - acc: 61.2% +- 1.19% - auc: 66.65% +- 2.31%

GAIN GLOBAL - acc: 61.36% +- 0.53% - auc: 66.85% +- 0.4%


## Run MICE

In [7]:
imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 61.7% +- 0.0% - auc: 67.3% +- 0.0%


## Run MISSFOREST

In [8]:
miss_accs, miss_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = MissForest(n_jobs=-1, random_state=i)
    imputed = imputer.fit_transform(data_missing_nans)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMISSFOREST RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    miss_accs.append(acc.mean())
    miss_aucs.append(auc.mean())
print(f'\nMISSFOREST GLOBAL - acc: {round(np.array(miss_accs).mean() * 100, 2)}% +- {round(np.array(miss_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(miss_aucs).mean() * 100, 2)}% +- {round(np.array(miss_aucs).std() * 100, 2)}%')

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4

MISSFOREST RUN 1/10 - acc: 61.85% +- 2.35% - auc: 67.39% +- 3.61%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3

MISSFOREST RUN 2/10 - acc: 61.35% +- 2.61% - auc: 66.65% +- 2.53%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3

MISSFOREST RUN 3/10 - acc: 63.4% +- 1.75% - auc: 67.61% +- 2.22%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3

MISSFOREST RUN 4/10 - acc: 62.6% +- 1.28% - auc: 67.1% +- 1.47%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4

MISSFOREST RUN 5/10 - acc: 60.6% +- 2.53% - auc: 65.69% +- 2.52%

Iteration: 0
Iteration: 1
Iteration: 2

MISSFOREST RUN 6/10 - acc: 60.5% +- 1.42% - auc: 65.91% +- 2.03%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 7/10 - acc: 61.95% +- 2.02% - auc: 67.24% +- 1.51%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 8/10 - acc: 60.2% +- 2.38% - auc

## Run SOFTIMPUTE

In [9]:
soft_accs, soft_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = SoftImpute(random_state=i)
    imputer.fit(data_missing_nans)
    imputed = imputer.predict(data_missing_nans)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'SOFTIMPUTE RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    soft_accs.append(acc.mean())
    soft_aucs.append(auc.mean())
print(f'\nSOFTIMPUTE GLOBAL - acc: {round(np.array(soft_accs).mean() * 100, 2)}% +- {round(np.array(soft_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(soft_aucs).mean() * 100, 2)}% +- {round(np.array(soft_aucs).std() * 100, 2)}%')

SOFTIMPUTE RUN 1/10 - acc: 62.15% +- 1.78% - auc: 67.33% +- 3.38%
SOFTIMPUTE RUN 2/10 - acc: 61.85% +- 2.58% - auc: 67.61% +- 3.12%
SOFTIMPUTE RUN 3/10 - acc: 62.05% +- 1.71% - auc: 66.61% +- 1.89%
SOFTIMPUTE RUN 4/10 - acc: 62.3% +- 2.67% - auc: 68.77% +- 3.56%
SOFTIMPUTE RUN 5/10 - acc: 62.2% +- 1.29% - auc: 67.6% +- 2.76%
SOFTIMPUTE RUN 6/10 - acc: 62.1% +- 1.39% - auc: 66.18% +- 1.82%
SOFTIMPUTE RUN 7/10 - acc: 61.75% +- 1.89% - auc: 67.3% +- 3.57%
SOFTIMPUTE RUN 8/10 - acc: 62.3% +- 1.83% - auc: 66.63% +- 2.41%
SOFTIMPUTE RUN 9/10 - acc: 59.05% +- 2.62% - auc: 64.42% +- 2.3%
SOFTIMPUTE RUN 10/10 - acc: 61.25% +- 1.43% - auc: 66.87% +- 2.36%

SOFTIMPUTE GLOBAL - acc: 61.7% +- 0.93% - auc: 66.93% +- 1.08%


## Run SINKHORN

In [10]:
sink_accs, sink_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = OTimputer(niter=500)
    imputed = imputer.fit_transform(data_missing_nans).cpu().detach().numpy()
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nSINKHORN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    sink_accs.append(acc.mean())
    sink_aucs.append(auc.mean())
print(f'\nSINKHORN GLOBAL - acc: {round(np.array(sink_accs).mean() * 100, 2)}% +- {round(np.array(sink_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(sink_aucs).mean() * 100, 2)}% +- {round(np.array(sink_aucs).std() * 100, 2)}%')

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.3529

SINKHORN RUN 1/10 - acc: 59.55% +- 2.04% - auc: 63.89% +- 2.17%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.3886

SINKHORN RUN 2/10 - acc: 61.75% +- 3.32% - auc: 65.5% +- 4.61%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.3466

SINKHORN RUN 3/10 - acc: 62.05% +- 2.43% - auc: 67.18% +- 2.32%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.4362

SINKHORN RUN 4/10 - acc: 59.1% +- 2.74% - auc: 64.02% +- 3.3%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.3403

SINKHORN RUN 5/10 - acc: 61.0% +- 1.71% - auc: 67.08% +- 1.98%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.3580

SINKHORN RUN 6/10 - acc: 59.95% +- 1.51% - auc: 64.55% +- 2.14%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.3399

SINKHORN RUN 7/10 - acc: 60.95% +- 3.05% - auc: 65.35% +- 4.6%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 1.3619

SINKHORN RUN 8/10 - acc: 60.4% +- 0.34% - auc: 64.47% 

## Run MIDA

In [11]:
mida_accs, mida_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = mida(data_missing_nans, num_layers=2, num_epochs=1500)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMIDA RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    mida_accs.append(acc.mean())
    mida_aucs.append(auc.mean())
print(f'\nMIDA GLOBAL - acc: {round(np.array(mida_accs).mean() * 100, 2)}% +- {round(np.array(mida_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(mida_aucs).mean() * 100, 2)}% +- {round(np.array(mida_aucs).std() * 100, 2)}%')

Stop training at epoch: 1500/1500, return best output

MIDA RUN 1/10 - acc: 62.8% +- 2.49% - auc: 67.54% +- 3.37%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 2/10 - acc: 62.85% +- 2.11% - auc: 68.36% +- 2.99%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 3/10 - acc: 63.2% +- 2.99% - auc: 66.69% +- 3.4%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 4/10 - acc: 64.65% +- 1.76% - auc: 69.74% +- 2.76%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 5/10 - acc: 62.2% +- 1.83% - auc: 65.35% +- 3.09%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 6/10 - acc: 62.0% +- 1.04% - auc: 67.48% +- 1.84%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 7/10 - acc: 61.55% +- 2.45% - auc: 66.01% +- 3.6%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 8/10 - acc: 61.0% +- 1.84% - auc: 66.19% +- 2.19%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 9/10 - acc: 61.

## Run T-tests

In [12]:
for model, metrics in {
        'MEDIAN': {'ACC': median_accs, 'AUC': median_aucs},
        'MEAN': {'ACC': mean_accs, 'AUC': mean_aucs},
        'KNN': {'ACC': knn_accs, 'AUC': knn_aucs},
        'GAIN': {'ACC': gain_accs, 'AUC': gain_aucs},
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'MISS': {'ACC': miss_accs, 'AUC': miss_aucs},
        'SOFT': {'ACC': soft_accs, 'AUC': soft_aucs},
        'SINK': {'ACC': sink_accs, 'AUC': sink_aucs},
        'MIDA': {'ACC': mida_accs, 'AUC': mida_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MEDIAN
Metric AUC - OUR METHOD is significantly better than MEDIAN
Metric ACC - OUR METHOD is significantly better than MEAN
Metric AUC - OUR METHOD is significantly better than MEAN
Metric ACC - OUR METHOD is significantly better than KNN
Metric AUC - OUR METHOD is significantly better than KNN
Metric ACC - OUR METHOD is significantly better than GAIN
Metric AUC - OUR METHOD is significantly better than GAIN
Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than MISS
Metric AUC - OUR METHOD is significantly better than MISS
Metric ACC - OUR METHOD is significantly better than SOFT
Metric AUC - OUR METHOD is significantly better than SOFT
Metric ACC - OUR METHOD is significantly better than SINK
Metric AUC - OUR METHOD is significantly better than SINK
Metric ACC - OUR METHOD is significantly better than MIDA
Metric AUC -